<a href="https://colab.research.google.com/github/JoyeBright/Sentiment-Analysis/blob/master/SentiPers/Classifier/NN/GoogleColab/Customized_CustomizedEmb/Customized_CustomizedEmb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install hazm
!pip install stopwords_guilannlp

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils.np_utils import to_categorical
from keras.utils import plot_model
from keras.backend import eval
from keras import optimizers
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale
import io

Using TensorFlow backend.


# Import dataset files

In [4]:
from google.colab import files
uploaded = files.upload()

Saving test_vecs_w2v.csv to test_vecs_w2v.csv
Saving train_vecs_w2v.csv to train_vecs_w2v.csv
Saving y_test.csv to y_test.csv
Saving y_train.csv to y_train.csv


# Conversion

In [6]:
y_train = pd.Series.from_csv("y_train.csv", sep="\t", header=0)
y_test = pd.Series.from_csv("y_test.csv", sep="\t", header=0)
test_vecs_w2v = pd.read_csv("test_vecs_w2v.csv", sep="\t")
train_vecs_w2v = pd.read_csv("train_vecs_w2v.csv", sep="\t")


test_vecs_w2v = test_vecs_w2v.iloc[:, 1:]
train_vecs_w2v = train_vecs_w2v.iloc[:, 1:]


test_vecs_w2v = test_vecs_w2v.values
train_vecs_w2v = train_vecs_w2v.values

# print(y_train[0])
# print(train_vecs_w2v[0])

# print(y_train.shape)
# print(y_test.shape)
# print(test_vecs_w2v.shape)
# print(train_vecs_w2v.shape)



/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


# Computation Environment

In [13]:
# CPU
!lscpu |grep 'Model name'

#no.of sockets i.e available slots for physical processors
!lscpu | grep 'Socket(s):'

#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:'

#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

#memory that we can use
!cat /proc/meminfo | grep 'MemAvailable'

#if it had turbo boost it would've shown Min and Max MHz also but it is only showing current frequency this means it always operates at 2.3GHz
!lscpu | grep "MHz"

#GPU count and name
!nvidia-smi -L

#use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'
!nvidia-smi

Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Socket(s):           1
Core(s) per socket:  1
Thread(s) per core:  2
MemAvailable:   12156288 kB
CPU MHz:             2300.000
GPU 0: Tesla K80 (UUID: GPU-0792bc69-8f1f-2163-d58a-75cacc7762e5)
Thu Jan 17 05:14:21 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    72W / 149W |    170MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
   

# Customized NN Model Using Customized Word Embedding

In [27]:
# Corpus has categorical value as its output so making categorical output is necessary
categorical_y_train = to_categorical(y_train, 5)
categorical_y_test = to_categorical(y_test, 5)

print(categorical_y_train.shape)
print(categorical_y_test.shape)

model = Sequential()
model.add(Dense(300, activation='relu', input_dim=100))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=300)
print('Train ...')
print('Learning Rate:',model.optimizer.lr)
model.fit(train_vecs_w2v, categorical_y_train,
          validation_data=(test_vecs_w2v, categorical_y_test),
          epochs=1000, batch_size=5561, callbacks=[early_stopping_monitor], verbose=2)
score = model.evaluate(test_vecs_w2v, categorical_y_test)
plot_model(model, show_shapes=True, to_file='CustomizedNN-CustomizedEmb.png')
print(score[1])


(5561, 5)
(1854, 5)
Train ...
Learning Rate: <tf.Variable 'Adam_16/lr:0' shape=() dtype=float32_ref>
Train on 5561 samples, validate on 1854 samples
Epoch 1/1000
 - 2s - loss: 1.6524 - acc: 0.0423 - val_loss: 1.5447 - val_acc: 0.4234
Epoch 2/1000
 - 0s - loss: 1.5486 - acc: 0.4256 - val_loss: 1.4694 - val_acc: 0.4234
Epoch 3/1000
 - 0s - loss: 1.4668 - acc: 0.4256 - val_loss: 1.3952 - val_acc: 0.4234
Epoch 4/1000
 - 0s - loss: 1.3898 - acc: 0.4256 - val_loss: 1.3371 - val_acc: 0.4234
Epoch 5/1000
 - 0s - loss: 1.3302 - acc: 0.4256 - val_loss: 1.3204 - val_acc: 0.4234
Epoch 6/1000
 - 0s - loss: 1.3156 - acc: 0.4256 - val_loss: 1.3181 - val_acc: 0.4234
Epoch 7/1000
 - 0s - loss: 1.3126 - acc: 0.4256 - val_loss: 1.3102 - val_acc: 0.4234
Epoch 8/1000
 - 0s - loss: 1.2973 - acc: 0.4256 - val_loss: 1.3110 - val_acc: 0.4234
Epoch 9/1000
 - 0s - loss: 1.2893 - acc: 0.4256 - val_loss: 1.3307 - val_acc: 0.4234
Epoch 10/1000
 - 0s - loss: 1.3006 - acc: 0.4256 - val_loss: 1.3324 - val_acc: 0.4234
